In [2]:
import numpy as np

## Create some data

In [3]:
data = [  # first two columns being features and last one being response
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]
header = ['color', 'diameter', 'label']

In [4]:
labels = []
for row in data:
    labels.append(row[0])
unique_labels = set(labels)
unique_labels

{'Green', 'Red', 'Yellow'}

## CART model

impurity: chance of being incorrect if randomly assign a label to an example in the same set

* What questions to ask?
* When to ask question?

To generate a list of questions, we'll iterate over every value for every feature in the data.

In [5]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [6]:
class Question:
    """A Question is used to partition a dataset."""
    
    def __init__(self, column, value):
        self.column = column  # column no (0 for color)
        self.value = value  # column value (e.g. Green)
        
    def match(self, example):
        """Compare feature value in Question to feature value in example"""
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        """Print Question"""
        condition = '=='
        if is_numeric(self.value):
            condition = '>='
        return 'Is ' + str(header[self.column]) + str(condition) + str(self.value) + '?'

In [7]:
def partition(rows, question):
    """For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'."""
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

The best question is the one which reduces the uncertainity the most. Gini impurity quantifies how much uncertainity is at a node. Information gain quantifies how much a question reduces the uncertainity. Entropy is a probalistic measure of uncertainity.
* Gini impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset
* Information gain (measure of reduction in uncertainity) is used to decide which feature to split on at each step in building the tree. Simplicity is best, so we want to keep our tree small. To do so, at each step we should choose the split that results in the purest daughter nodes.

To compute Gini impurity for a set of items with $J$ classes, suppose $i \in { 1 , 2 , . . . , J }$, and let $p_i$ be the fraction of items labeled with class $i$ in the set:  
[Gini impurity](https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity) = $\sum($P(item with label $i$) * P(mistake in  categorizing that label $i$)$)$
$$I_G = 1 - \sum_{i=1}^J p_i^2$$

In [8]:
def class_counts(rows):
    """Counts number of each type of example in the dataset"""
    counts = {}
    for row in rows:
        label = row[-1]  # last column
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [9]:
def gini(rows):
    """Calculate Gini impurity for list of rows"""
    counts = class_counts(rows)
    impurity = 1
    for label in counts:
        prob_label = counts[label] / float(len(rows))
        impurity -= prob_label ** 2
    return impurity

In [10]:
def info_gain(left, right, current_uncertainity):
    """Information gain: the uncertainty of the starting node, minus the 'weighted' (average) impurity of
    two child nodes."""
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainity - p*gini(left) - (1 - p)*gini(right)

In [11]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of information gain
    best_question = None  # keep track of feature/value that produced it
    current_uncertainity = gini(rows)
    n_features = len(rows[0]) - 1  # no of columns
    
    for col in range(n_features):  # for each feature
        values = set([row[col] for row in rows])  # unique values in column
        
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows)  == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainity)
            if gain >= best_gain:
                best_gain = gain
                best_question = question
    return best_gain, best_question

In [12]:
class Leaf:
    """A leaf node classifies data"""
    def __init__(self, rows):
        self.predictions = class_counts(rows)

class DecisionNode:
    """A decision node asks a question"""
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [13]:
def build_tree(rows):
    
    # artitioing the dataset on each of the unique attribute, calculate the information gain,
    # and return the question that produces the highest gain.
    info, question = find_best_split(rows)
    
    # base case (no information gain)
    if info == 0:
        return Leaf(rows)
    
    true_rows, false_rows = partition(rows, question)
    
    # recursive call to build true_branch and false_branch
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    # Return a Question node, which records the best feature / value to ask at this point
    return DecisionNode(question, true_branch, false_branch)

In [53]:
def print_tree(node, spacing=''):
    
    # base case (leaf)
    if isinstance(node, Leaf):
        print(spacing + 'Predict' + str(node.predictions))
        return
    
    # print question at this node
    print(spacing + str(node.question))
    
    # recursively print questions
    print(spacing + '---> True:')
    print_tree(node.true_branch, spacing + '\t')
    print(spacing + '---> False:')
    print_tree(node.false_branch, spacing + '\t')

In [54]:
tree = build_tree(data)
print_tree(tree)

Is diameter>=3?
---> True:
	Is color==Yellow?
	---> True:
		Predict{'Apple': 1, 'Lemon': 1}
	---> False:
		Predict{'Apple': 1}
---> False:
	Predict{'Grape': 2}


In [27]:
def classify(row, node):
    
    # base case
    if isinstance(node, Leaf):
        return node.predictions
    
    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [39]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for label in counts.keys():
        probs[label] = str(int(counts[label] / total * 100)) + '%'
    return probs

In [41]:
print_leaf(classify(data[1], tree))

{'Apple': '50%', 'Lemon': '50%'}

In [42]:
test = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [43]:
for row in test:
    print('Actual: {} \t Predicted: {}'.format(row[-1], print_leaf(classify(row, tree))))

Actual: Apple 	 Predicted: {'Apple': '100%'}
Actual: Apple 	 Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape 	 Predicted: {'Grape': '100%'}
Actual: Grape 	 Predicted: {'Grape': '100%'}
Actual: Lemon 	 Predicted: {'Apple': '50%', 'Lemon': '50%'}
